
# 📌 Aula 10: Criando um Chatbot Empresarial Pronto para Produção

Neste notebook, construiremos um chatbot seguro, escalável e pronto para uso empresarial.
Vamos implementar:

1. Segurança e autenticação usando JWT.
2. Armazenamento de conversas usando SQLite.
3. Comunicação com a API da OpenAI.
4. Implementação de um servidor Flask para expor o chatbot via API.

---


## 📦 Configuração do Ambiente

Instalaremos e importaremos as bibliotecas necessárias.

In [ ]:

# Instale as bibliotecas necessárias (se ainda não estiverem instaladas)
!pip install openai flask jwt sqlite3 requests logging


In [ ]:

# Importação das bibliotecas
import openai
import jwt
import logging
import sqlite3
import requests
from flask import Flask, request, jsonify

# Configuração do log para facilitar o debug
logging.basicConfig(level=logging.INFO)


## 🗄️ Configuração do Banco de Dados

Vamos criar um banco SQLite para armazenar as conversas do chatbot.

In [ ]:

# Configuração do banco de dados SQLite
conn = sqlite3.connect('chatbot.db', check_same_thread=False)
cursor = conn.cursor()
cursor.execute('''CREATE TABLE IF NOT EXISTS conversas (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    usuario TEXT,
    mensagem TEXT,
    resposta TEXT,
    timestamp DATETIME DEFAULT CURRENT_TIMESTAMP
)''')
conn.commit()


## 🔑 Implementando Autenticação com JWT

Usaremos JSON Web Tokens (JWT) para autenticação segura.

In [ ]:

# Função para gerar tokens JWT
def gerar_token(usuario):
    token = jwt.encode({"user": usuario}, "chave_secreta", algorithm="HS256")
    return token

# Testando geração de token
print("Token gerado:", gerar_token("admin"))


## 💾 Armazenando Conversas

Cada interação do usuário será salva no banco de dados.

In [ ]:

# Função para armazenar conversas no banco de dados
def salvar_conversa(usuario, mensagem, resposta):
    cursor.execute("INSERT INTO conversas (usuario, mensagem, resposta) VALUES (?, ?, ?)", (usuario, mensagem, resposta))
    conn.commit()


## 🤖 Comunicação com OpenAI API

Agora conectaremos o chatbot à API da OpenAI para processar respostas.

In [ ]:

# Função para obter resposta do OpenAI API
def obter_resposta_openai(mensagem):
    response = openai.ChatCompletion.create(
        model="gpt-4",
        messages=[{"role": "system", "content": "Você é um assistente empresarial."},
                  {"role": "user", "content": mensagem}]
    )
    return response['choices'][0]['message']['content']


## 🌍 Criando uma API com Flask

Agora, vamos expor o chatbot como uma API para interações via requisições HTTP.

In [ ]:

# Configuração do Flask
app = Flask(__name__)

# Rota para interagir com o chatbot
@app.route("/chat", methods=["POST"])
def chat():
    data = request.json
    usuario = data.get("usuario")
    mensagem = data.get("mensagem")
    
    if not usuario or not mensagem:
        return jsonify({"erro": "Usuário e mensagem são obrigatórios"}), 400
    
    resposta = obter_resposta_openai(mensagem)
    salvar_conversa(usuario, mensagem, resposta)
    
    return jsonify({"resposta": resposta})

# Rodando a aplicação Flask
if __name__ == "__main__":
    app.run(debug=True)
